## Keyword Tweet Word Cloud (Windows OS) ##

A program for generating tweet based word clouds. Specifically in regards to the _cleaned_, geo-tagged Twitter data from April 12 to 22 of 2013.

The program may be ammended to create any number of word clouds given an input txt file. A csv file may be used as well, the relavent information from which is parsed and saved into a txt file for use by the word cloud function.

In [ ]:
# Author: Elizabeth Brooks
# Date Modified: 06/25/2015
# Edited: Hayden Fuss

# PreProcessor Directives
%matplotlib inline
import matplotlib
import csv
import matplotlib.pyplot as plt
# Make sure to use < pip install pillow >
# With anaconda, use < conda install pil >
from wordcloud import WordCloud
from scipy.misc import imread
import re
# Add parent directory to path for twc imports
import os
import sys
sys.path.append(os.path.realpath('../'))
import twitter_criteria as twc

# Global field declarations
twc.loadCriteria()
keyword = twc.getKeywordRegex()
twc.clearCriteria()

# Function to clean up tweet strings 
# by manually removing irrelevant data (not words)
def cleanUpTweet(tweet):
    # Irrelevant characters
    twitter_markup = ['&amp;', 'http://t.co/']
    temp = tweet.lower()
    # Use regex to create a regular expression 
    # for removing undesired characters
    temp = re.sub('|'.join(twitter_markup), r"", temp)
    return temp
# End cleanUpTweet
## The markup and the cleanUpTweet function will eventually 
## be moved to twc

# Function to search for tweets wased on pre-determined key words
def tweetHasAKeyword(tweet):
    return keyword.search(tweet) is not None
# End tweetHasAKeyword


The function .cleanUpTweet(String) is used to remove irrelevant data, characters that are not words and are therefore not interesting. In order to produce a more refined word cloud, the .tweetHasAKeyword(String) searches the data set of cleaned tweet strings for those tweets containing our pre-determined set of key words (see below).

keywords: ["#bostonmarathon",
           "#marathonmonday",
           "#patriotsday",
           "marathon",
           "boylston",
           "finish line",
           "#bostonstrong",
           "#bostonpride",
           "#prayforboston",
           "#pray4bos",
           "bomb",
           "explosion",
           "explode",
           "wounded",
           "hostage",
           "watertown",
           "lockdown",
           "manhunt",
           "collier"]

In [ ]:
# Function for generating a word cloud
## Windows OS specific font path in arguments
### On Windows, if the given file name does not exist, 
### the loader also looks in Windows fonts directory
def tweetWordCloud(filepath, maskpath=None, bg_color='white', num_words=150,
                   fontpath='sans-serif.ttf', outpath='keywordCloud.png'):
    # Read in the txt file set by the main method
    text = open(filepath, 'r').read()
    # Generate the word cloud based on the input arguments
    wc = None
    if maskpath:
        # Use module to read the image file
        mass_mask = imread(maskpath)
        wc = WordCloud(mask=mass_mask, background_color=bg_color, max_words=num_words, 
                       font_path=fontpath).generate(text)
    else:
        wc = WordCloud(font_path=fontpath, max_words=num_words,
                       background_color=bg_color).generate(text)
    # Open a plot of the generated word cloud
    wc.to_file(outpath)
    plt.imshow(wc)
    plt.axis("off")
    plt.show()
# End tweetWordCloud
## We can eventually move this to its own module


The above function generates a word cloud based on words in the txt file created in the main method.
For more info see: https://github.com/amueller/word_cloud

In [ ]:
# The main method
def main():
    # Set the output image file path
    current_dir = os.getcwd()
    filepath = current_dir + '/OutputTweets.txt'    # Create object for writting to a text file
    tweetFile = open(filepath, "w")
    # Iterate through the "cleaned" Twitter data by tweet
    with open(current_dir + '/../cleaned_geo_tweets_Apr_12_to_22.csv') as csvfile:  
        tweetIt = csv.DictReader(csvfile)
        # Retrieve the strings of tweets
        for twitterData in tweetIt:
            # Convert tweets to lower case to pool words of the same spelling
            tweet_text = cleanUpTweet(twitterData['tweet_text'])
            # Write the selected Twitter data, tweets, to the txt file
            if tweetHasAKeyword(tweet_text):
                tweetFile.write(tweet_text)
    # Close the file obj
    tweetFile.close()  
    # Use the defined function to create the tweet word cloud
    # in the shape of Massachusets
    tweetWordCloud(filepath, current_dir + '/mass_map.png')
# End main

# Run the script via the main method
if __name__ == "__main__":
    main()
    
# End script

The above code creates a file object and opens a txt file to store all the strings of tweets contained in the Twitter data from the "cleaned" tweets csv file. The tweet strings are then retrieved from the csv file by iterating through the tweet hashes made by the DictReader. Finally, these strings are converted to lower case. This is because we are interested in looking at the occurances of a given spelling of a word, regardless of case; stemming may be considered in the future to combine the frequencies of all conjugate forms of a words.